Modified from polychrom extrusion_3D\
Original Source : https://github.com/open2c/polychrom.git \
Licensed under MIT License\
Copyright (c) 2019 Massachusetts Institute of Technology

In [9]:
import pickle
import os
import time
import numpy as np
import polychrom

from polychrom import polymerutils
from polychrom import forces
from polychrom import forcekits
from polychrom.simulation import Simulation
from polychrom.starting_conformations import grow_cubic
from polychrom.hdf5_format import HDF5Reporter, list_URIs, load_URI, load_hdf5_file

import openmm 
import os 
import shutil


import warnings
import h5py 
import glob

import logging
import sys
from tqdm import tqdm
from IPython.display import clear_output

In [10]:
from bondUpdater import bondUpdater

In [11]:
input_folder_name = "trajectory_20260224_1"
h5_filepath = f"{input_folder_name}/LEFPositions_big.h5"
output_folder_name = "LEFPositions_simu_20260224_big_chromosome"

In [12]:
def add_polymer_physics(sim, is_ring=False, trunc=1.5, angle_k=1.5, bond_length=1.0, bond_wiggle=0.1):
    """
    polymer_physics
    """
    sim.add_force(
        forcekits.polymer_chains(
            sim,
            chains=[(0, None, is_ring)],
            bond_force_func=forces.harmonic_bonds,
            bond_force_kwargs={
                'bondLength': bond_length,
                'bondWiggleDistance': bond_wiggle,
            },
            angle_force_func=forces.angle_force,
            angle_force_kwargs={
                'k': angle_k
            },
            nonbonded_force_func=forces.polynomial_repulsive,
            nonbonded_force_kwargs={
                'trunc': trunc, 
                'radiusMult': 1.05,
            },
            except_bonds=True,
        )
    )

In [13]:
def make_chains_eq(N, number_of_chains, is_ring):
    if number_of_chains == 1:
        return [(0, None, is_ring)]
    #for multiple chains
    return [(i * (N // number_of_chains), (i + 1) * (N // number_of_chains), is_ring) 
            for i in range(number_of_chains)]

In [14]:
logging.getLogger("polychrom").setLevel(logging.WARNING)

def run_experiment(folder_name, h5_filepath, params):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    with h5py.File(h5_filepath, mode='r') as f:
        lef_positions = f["positions"][:]
        N = f.attrs["N"]

    milker = bondUpdater(lef_positions)
    reporter = HDF5Reporter(folder=folder_name, max_data_length=100, overwrite=True)
    
    data = params['initial_data']
    sim_inits_total = lef_positions.shape[0] // params['restart_every']

    pbar = tqdm(total=sim_inits_total, desc="Total Progress")
    
    #main simulation loop
    for iteration in range(sim_inits_total):

        clear_output(wait=True)

        pbar.display()
        print(f"Current Progress: {iteration + 1} / {sim_inits_total} calculation in progress...")
        
        sim = Simulation(
            platform="cuda", GPU="0", integrator=params.get('integrator','variableLangevin'),
            error_tol=0.01, N=N, reporters=[reporter],
            PBCbox=params['box'], precision="mixed", collision_rate=params.get('collision_rate', 0.03), 
            temperature=params.get('temperature', 300),
            verbose=False
        ) #you can actually change the parameters
        
        sim.set_data(data)

        add_polymer_physics(
            sim, 
            angle_k=params.get('angle_k', 1.5),
            trunc=params.get('trunc', 1.5)
        )

        kbond = sim.kbondScalingFactor / (params['smc_wiggle'] ** 2)
        milker.setParams({"length": params['smc_dist'], "k": kbond}, 
                         {"length": params['smc_dist'], "k": 0})
        milker.setup(bondForce=sim.force_dict['harmonic_bonds'], blocks=params['restart_every'])

        if iteration == 0: sim.local_energy_minimization()
        else: sim._apply_forces()

        #calculation loop
        for i in range(params['restart_every']):
            if i % params['save_every'] == (params['save_every'] - 1):
                sim.do_block(steps=params['steps'])
            else:
                sim.integrator.step(params['steps'])
            
            if i < params['restart_every'] - 1:
                milker.step(sim.context)

        data = sim.get_data()
        del sim
        reporter.blocks_only = True

        pbar.update(1)
        time.sleep(0.2)

    pbar.close()
    reporter.dump_data()


Run the Simulation!

Basic Parameters

In [15]:
# -------defining parameters----------
#  -- basic loop extrusion parameters

myfile = h5py.File(h5_filepath, mode='r') #you have to create your "trajectory" folder and "LEFPosition.h5" in order to run the simulation

N = myfile.attrs["N"]
LEFNum = myfile.attrs["LEFNum"]
LEFpositions = myfile["positions"]

Nframes = LEFpositions.shape[0] #check the frames and get how many frames does LEFpositions have

    
steps = 750   # MD steps per step of cohesin
stiff = 1
dens = 0.1
box = (N / dens) ** 0.33  # density = 0.1.
data = grow_cubic(N, int(box) - 2)  # creates a compact conformation ~ for reference, see the polychrom.starting_conformations
block = 0  # starting block 

# new parameters because some things changed 
saveEveryBlocks = 10   # save every 10 blocks (saving every block is now too much almost)
restartSimulationEveryBlocks = 100

# parameters for smc bonds
smcBondWiggleDist = 0.2
smcBondDist = 0.5

# assertions for easy managing code below 
assert (Nframes % restartSimulationEveryBlocks) == 0 
assert (restartSimulationEveryBlocks % saveEveryBlocks) == 0

savesPerSim = restartSimulationEveryBlocks // saveEveryBlocks
simInitsTotal  = (Nframes) // restartSimulationEveryBlocks

In [16]:
if __name__ == "__main__":
    my_params = {
        'initial_data': data, # grow_cubic 등으로 생성한 데이터
        'box': [100, 100, 100],
        'restart_every': 100, #calculates 100 steps and updates ~ cohesin one move, polymer how many move
        'save_every': 100,
        'steps': 1000, #total blocks, steps
        'smc_wiggle': 0.2,
        'smc_dist': 0.5,
        'angle_k': 1.5,
        'collision_rate' : 0.03,
        'temperature' : 300,
        'integrator' : 'variableLangevin',
    }
    
    run_experiment(output_folder_name, h5_filepath, my_params)


Total Progress:  99%|███████████████████████████████████████████████████████▍| 99/100 [1:45:05<01:02, 62.92s/it]INFO:root:adding force harmonic_bonds 0
INFO:root:adding force angle 1
INFO:root:Using periodic boundary conditions
INFO:root:adding force polynomial_repulsive 2


Current Progress: 100 / 100 calculation in progress...
Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 49999


INFO:root:Particles loaded. Potential energy is 1.285230
INFO:root:block    0 pos[1]=[55.2 -19.5 111.3] dr=19.67 t=17041.7ps kin=1.46 pot=1.25 Rg=50.011 SPS=1558 dt=170.4fs dx=46.07pm 

Total Progress: 100%|███████████████████████████████████████████████████████| 100/100 [1:46:11<00:00, 63.71s/it]


In [59]:
#check for iterations
with h5py.File(h5_filepath, mode='r') as f:
    total_lef_steps = f["positions"].shape[0]

print(f"Total LEF Steps: {total_lef_steps}")
print(f"Restart Every: {my_params['restart_every']}")
print(f"Total Iteration: {total_lef_steps // my_params['restart_every']}")

Total LEF Steps: 10000
Restart Every: 100
Total Iteration: 100
